## triplet

In [1]:
import json

with open("dataset/rag_truth_train.json", "r") as f:
    train_data = json.load(f)
with open("dataset/rag_truth_dev.json", "r") as f:
    dev_data = json.load(f)
with open("dataset/rag_truth_test.json", "r") as f:
    test_data = json.load(f)

In [2]:
def add_prefix(data):
    for d in data:
        d["text"] = "Please judge the following statement whether it includes hallucination or not based on the references above: " + d["text"]
    return data

train_data = add_prefix(train_data)
dev_data = add_prefix(dev_data)
test_data = add_prefix(test_data)

In [3]:
# task_type: QA, Data2txt, Summary
task_name = "Summary"
train_data = [d for d in train_data if d["task_type"] == task_name]
dev_data = [d for d in dev_data if d["task_type"] == task_name]
test_data = [d for d in test_data if d["task_type"] == task_name]

In [3]:
import random

def create_trip(data, id_list):
    trip = []
    for id in id_list:
        num = 0
        no_hal = []
        has_hal = []
        for d in data:
            if num == 6:
                num = 0
                if no_hal == [] or has_hal == []:
                    break
                # shuffle
                random.seed(id) # change seed
                no_hal = random.sample(no_hal, len(no_hal))
                has_hal = random.sample(has_hal, len(has_hal))
                
                if len(no_hal)==1 or len(no_hal)==5:
                    trip.append({"anchor":ref,"positive": no_hal[0], "negative": has_hal[0], "labels": 0}) # labels are dummy 
                elif len(no_hal)==2 or len(no_hal)==4:
                    trip.append({"anchor":ref,"positive": no_hal[0], "negative": has_hal[0], "labels": 0})
                    trip.append({"anchor":ref,"positive": no_hal[1], "negative": has_hal[1], "labels": 0})
                elif len(no_hal)==3:
                    trip.append({"anchor":ref,"positive": no_hal[0], "negative": has_hal[0], "labels": 0})
                    trip.append({"anchor":ref,"positive": no_hal[1], "negative": has_hal[1], "labels": 0})
                    trip.append({"anchor":ref,"positive": no_hal[2], "negative": has_hal[2], "labels": 0})
                no_hal = []
                has_hal = []
                break
            elif d["source_id"] == id:
                num +=1
                ref = d["ref"]
                if d["labels"] == 0:
                    no_hal.append(d["text"])
                else: #hallucination
                    has_hal.append(d["text"])
        if num == 6:
            if len(no_hal)==1 or len(no_hal)==5:
                trip.append({"anchor":ref,"positive": no_hal[0], "negative": has_hal[0], "labels": 0})
            elif len(no_hal)==2 or len(no_hal)==4:
                trip.append({"anchor":ref,"positive": no_hal[0], "negative": has_hal[0], "labels": 0})
                trip.append({"anchor":ref,"positive": no_hal[1], "negative": has_hal[1], "labels": 0})
            elif len(no_hal)==3:
                trip.append({"anchor":ref,"positive": no_hal[0], "negative": has_hal[0], "labels": 0})
                trip.append({"anchor":ref,"positive": no_hal[1], "negative": has_hal[1], "labels": 0})
                trip.append({"anchor":ref,"positive": no_hal[2], "negative": has_hal[2], "labels": 0})
    return trip

In [4]:
train_id = [d["source_id"] for d in train_data]
train_id = list(set(train_id))
dev_id = [d["source_id"] for d in dev_data]
dev_id = list(set(dev_id))
test_id = [d["source_id"] for d in test_data]
test_id = list(set(test_id))
print(len(train_id), len(dev_id), len(test_id))
train_trip = create_trip(train_data, train_id)
dev_trip = create_trip(dev_data, dev_id)
test_trip = create_trip(test_data, test_id)

2305 210 450


In [5]:
len(train_trip), len(dev_trip), len(test_trip)

(3760, 337, 633)

In [6]:
from datasets import Dataset, DatasetDict
import pandas as pd

train_df = pd.DataFrame(train_trip)
dev_df = pd.DataFrame(dev_trip)
test_df = pd.DataFrame(test_trip)

train_ds = Dataset.from_pandas(train_df)
dev_ds = Dataset.from_pandas(dev_df)
test_ds = Dataset.from_pandas(test_df)

tri_raw_datasets = DatasetDict({"train": train_ds, "dev": dev_ds, "test": test_ds})
tri_raw_datasets

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive', 'negative', 'labels'],
        num_rows: 3760
    })
    dev: Dataset({
        features: ['anchor', 'positive', 'negative', 'labels'],
        num_rows: 337
    })
    test: Dataset({
        features: ['anchor', 'positive', 'negative', 'labels'],
        num_rows: 633
    })
})

In [7]:
from transformers import AutoTokenizer

tri_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")


In [8]:
tri_raw_datasets["train"][0]

{'anchor': '{\'name\': \'Meet Up Chinese Cuisine\', \'address\': \'2251 Las Positas Rd\', \'city\': \'Santa Barbara\', \'state\': \'CA\', \'categories\': \'Szechuan, Chinese, Nightlife, Karaoke, Asian Fusion, Restaurants\', \'hours\': {\'Monday\': \'11:0-22:0\', \'Tuesday\': \'11:0-22:0\', \'Wednesday\': \'11:0-22:0\', \'Thursday\': \'11:0-22:0\', \'Friday\': \'11:0-22:0\', \'Saturday\': \'11:0-22:0\', \'Sunday\': \'11:0-22:0\'}, \'attributes\': {\'BusinessParking\': {\'garage\': False, \'street\': True, \'validated\': False, \'lot\': True, \'valet\': False}, \'RestaurantsReservations\': True, \'OutdoorSeating\': True, \'WiFi\': \'free\', \'RestaurantsTakeOut\': True, \'RestaurantsGoodForGroups\': True, \'Music\': True, \'Ambience\': {\'divey\': True, \'hipster\': None, \'casual\': True, \'touristy\': None, \'trendy\': True, \'intimate\': None, \'romantic\': None, \'classy\': True, \'upscale\': None}}, \'business_stars\': 4.0, \'review_info\': [{\'review_stars\': 5.0, \'review_date\': 

In [9]:
from transformers import DataCollatorWithPadding

def tri_tokenize_function(examples):
    anchor = tri_tokenizer(examples["anchor"], truncation=True,max_length=512)
    positive = tri_tokenizer(examples["positive"], truncation=True,max_length=512)
    negative = tri_tokenizer(examples["negative"], truncation=True,max_length=512)

    return {
        "anchor_input_ids": anchor["input_ids"],
        "anchor_attention_mask": anchor["attention_mask"],
        "positive_input_ids": positive["input_ids"],
        "positive_attention_mask": positive["attention_mask"],
        "negative_input_ids": negative["input_ids"],
        "negative_attention_mask": negative["attention_mask"],
    }

tri_tokenized_datasets = tri_raw_datasets.map(tri_tokenize_function, batched=True)
tri_tokenized_datasets = tri_tokenized_datasets.remove_columns(["anchor", "positive", "negative"])
tri_tokenized_datasets.set_format("torch")
tri_data_collator = DataCollatorWithPadding(tokenizer=tri_tokenizer)

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

Map:   0%|          | 0/337 [00:00<?, ? examples/s]

Map:   0%|          | 0/633 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorWithPadding
from torch.nn.utils.rnn import pad_sequence
import torch

class CustomDataCollator(DataCollatorWithPadding):
    def __call__(self, features):
        anchor_ids = [x['anchor_input_ids'].clone().detach() for x in features]
        positive_ids = [x['positive_input_ids'].clone().detach() for x in features]
        negative_ids = [x['negative_input_ids'].clone().detach() for x in features]
        
        anchor_mask = [x['anchor_attention_mask'].clone().detach() for x in features]
        positive_mask = [x['positive_attention_mask'].clone().detach() for x in features]
        negative_mask = [x['negative_attention_mask'].clone().detach() for x in features]
        
        anchor_ids = pad_sequence(anchor_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        positive_ids = pad_sequence(positive_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        negative_ids = pad_sequence(negative_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        
        anchor_mask = pad_sequence(anchor_mask, batch_first=True, padding_value=0)
        positive_mask = pad_sequence(positive_mask, batch_first=True, padding_value=0)
        negative_mask = pad_sequence(negative_mask, batch_first=True, padding_value=0)

        labels = [x['labels'] for x in features]
        
        batch = {
            "input_ids": [anchor_ids, positive_ids, negative_ids],
            "attention_mask": [anchor_mask, positive_mask, negative_mask],
            "labels": labels
        }
        
        return batch


tri_data_collator = CustomDataCollator(tokenizer=tri_tokenizer)

In [11]:
from transformers import AutoModel

base_model = AutoModel.from_pretrained("microsoft/Phi-3.5-mini-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
base_model.to(device)
device

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


device(type='cuda')

In [13]:
import torch.nn.functional as F
import torch.nn as nn

def triplet_loss(anchor_output, positive_output, negative_output, positive_logits, negative_logits):
    positive_targets = torch.zeros(positive_output.size(0), dtype=torch.long).to(device)  # no hallucination = 0
    negative_targets = torch.ones(negative_output.size(0), dtype=torch.long).to(device)
    positive_loss = nn.CrossEntropyLoss()(positive_logits, positive_targets)
    negative_loss = nn.CrossEntropyLoss()(negative_logits, negative_targets)

    classification_loss = (positive_loss + negative_loss) / 2.0 # average

    triplet_loss_fn = (nn.TripletMarginWithDistanceLoss(margin=1.0,distance_function=lambda x, y: 1.0 - F.cosine_similarity(x, y)))
    triplet_loss = triplet_loss_fn(anchor_output, positive_output, negative_output)
   
    return classification_loss, triplet_loss

In [14]:
import torch
import numpy as np

def compute_tri_metrics(eval_pred):
    logits,labels= eval_pred
    logits = logits[0]
   
    # ModelOutput(logits=[positive_logits, negative_logits]...)
    positive_logits = torch.tensor(logits[0])
    negative_logits = torch.tensor(logits[1])

    positive_preds = torch.argmax(positive_logits, dim=1)
    negative_preds = torch.argmax(negative_logits, dim=1)

    # num of correct predictions
    correct_positive = (positive_preds == 0).sum().item()
    correct_negative = (negative_preds == 1).sum().item()
    
    # total samples
    total_samples = positive_preds.size(0) + negative_preds.size(0)
    
    positive_preds_num = (positive_preds == 1).sum().item() # predict non-hallucination samples (positive) as hallucination
    negative_preds_num = (negative_preds == 1).sum().item() # predict hallucination samples (negative) as hallucination
    negative_num = negative_preds.size(0) # num of hallucination samples

    precision = negative_preds_num / (positive_preds_num + negative_preds_num) if (positive_preds_num + negative_preds_num) > 0 else 0
    recall = negative_preds_num / negative_num if negative_num > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    accuracy = (correct_positive + correct_negative) / total_samples
    return {
        "accuracy": accuracy,
        "recall": recall,
        "precision": precision,
        "f1": f1
    }
    


In [15]:
from transformers import TrainingArguments
from transformers import Trainer
import torch
from models_phi import TripletModel

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="steps",  
    save_steps=10000,
    learning_rate=1e-6,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16 = True,
    gradient_accumulation_steps=12,
    logging_dir="./logs",
    remove_unused_columns=False,
    report_to="tensorboard",
    optim="adafactor",
)

tri_model = TripletModel(base_model, triplet_loss)

trainer = Trainer(
    model=tri_model,
    args=training_args,
    train_dataset=tri_tokenized_datasets["train"],
    eval_dataset=tri_tokenized_datasets["dev"],
    data_collator=tri_data_collator,
    tokenizer=tri_tokenizer,
    compute_metrics=compute_tri_metrics,

)

/gs/fs/tga-arase-student/yamada/.venv/lib64/python3.9/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/3388348.1.all.q/ipykernel_3679849/2642719111.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:

trainer.evaluate()

{'eval_loss': 1.7633545398712158,
 'eval_model_preparation_time': 0.0042,
 'eval_accuracy': 0.4970326409495549,
 'eval_recall': 0.658753709198813,
 'eval_precision': 0.4977578475336323,
 'eval_f1': 0.5670498084291188,
 'eval_runtime': 12.357,
 'eval_samples_per_second': 27.272,
 'eval_steps_per_second': 6.879}

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,Recall,Precision,F1
1,1.483000,1.180639,0.004200,0.701780,0.735905,0.688889,0.711621
2,1.132300,1.079993,0.004200,0.731454,0.804154,0.702073,0.749654
3,1.015100,1.096025,0.004200,0.721068,0.700297,0.730650,0.715152
4,0.935900,1.035825,0.004200,0.738872,0.777448,0.721763,0.748571
5,0.842400,1.020121,0.004200,0.740356,0.771513,0.726257,0.748201
6,0.756800,1.035631,0.004200,0.759644,0.821958,0.730871,0.773743
7,0.687700,1.058635,0.004200,0.737389,0.747774,0.732558,0.740088
8,0.620300,1.039975,0.004200,0.753709,0.789318,0.736842,0.762178
9,0.508300,1.060465,0.004200,0.749258,0.744807,0.751497,0.748137


TrainOutput(global_step=780, training_loss=0.8576747943193485, metrics={'train_runtime': 4383.8, 'train_samples_per_second': 8.577, 'train_steps_per_second': 0.178, 'total_flos': 0.0, 'train_loss': 0.8576747943193485, 'epoch': 9.880851063829788})

In [18]:
trainer.evaluate(eval_dataset=tri_tokenized_datasets["test"])

{'eval_loss': 1.1320427656173706,
 'eval_model_preparation_time': 0.0042,
 'eval_accuracy': 0.7464454976303317,
 'eval_recall': 0.7424960505529226,
 'eval_precision': 0.7484076433121019,
 'eval_f1': 0.7454401268834259,
 'eval_runtime': 21.2358,
 'eval_samples_per_second': 29.808,
 'eval_steps_per_second': 7.487,
 'epoch': 9.880851063829788}

In [19]:
def create_dev_task(name):
    dev_data2 = [d for d in test_data if d["task_type"] == name]
    dev_id2 = [d["source_id"] for d in dev_data2]
    dev_id2 = list(set(dev_id2))
    dev_trip2 = create_trip(dev_data2, dev_id2)
    dev_df2 = pd.DataFrame(dev_trip2)
    dev_ds2 = Dataset.from_pandas(dev_df2)
    tri_tokenized_datasets_task = dev_ds2.map(tri_tokenize_function, batched=True)
    tri_tokenized_datasets_task = tri_tokenized_datasets_task.remove_columns(["anchor", "positive", "negative"])
    tri_tokenized_datasets_task.set_format("torch")
    return tri_tokenized_datasets_task


In [20]:
dev_qa = create_dev_task("QA")
trainer.evaluate(eval_dataset=dev_qa)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

{'eval_loss': 1.143991470336914,
 'eval_model_preparation_time': 0.0042,
 'eval_accuracy': 0.7266666666666667,
 'eval_recall': 0.7266666666666667,
 'eval_precision': 0.7266666666666667,
 'eval_f1': 0.7266666666666666,
 'eval_runtime': 4.7458,
 'eval_samples_per_second': 31.607,
 'eval_steps_per_second': 8.007,
 'epoch': 9.880851063829788}

In [21]:
dev_d2t = create_dev_task("Data2txt")
trainer.evaluate(eval_dataset=dev_d2t)

Map:   0%|          | 0/291 [00:00<?, ? examples/s]

{'eval_loss': 0.8743554353713989,
 'eval_model_preparation_time': 0.0042,
 'eval_accuracy': 0.8213058419243986,
 'eval_recall': 0.8694158075601375,
 'eval_precision': 0.7931034482758621,
 'eval_f1': 0.8295081967213116,
 'eval_runtime': 10.4336,
 'eval_samples_per_second': 27.891,
 'eval_steps_per_second': 6.997,
 'epoch': 9.880851063829788}

In [22]:
dev_sum = create_dev_task("Summary")
trainer.evaluate(eval_dataset=dev_sum)

Map:   0%|          | 0/192 [00:00<?, ? examples/s]

{'eval_loss': 1.5093679428100586,
 'eval_model_preparation_time': 0.0042,
 'eval_accuracy': 0.6432291666666666,
 'eval_recall': 0.5416666666666666,
 'eval_precision': 0.6797385620915033,
 'eval_f1': 0.6028985507246376,
 'eval_runtime': 5.9729,
 'eval_samples_per_second': 32.145,
 'eval_steps_per_second': 8.036,
 'epoch': 9.880851063829788}

In [23]:
name = "./trained/triplet_phi"
trainer.save_model(name)
trainer.save_state()
tri_model.save_pretrained(name)